## Model Training Callbacks

You can use the `DreadnodeCallback` from the `integrations` submodule to connect with the `Trainer` class from the `transformers` library. Model training can be tracked directly in Strikes.


In [ ]:
import dreadnode as dn

dn.configure(
    token="<YOUR API KEY>", # Replace with your token
)

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load dataset
dataset = load_dataset("glue", "sst2")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Take a small portion of the dataset
dataset["train"] = dataset["train"].select(range(1000))
dataset["validation"] = dataset["validation"].select(range(1000))

# Preprocessing function
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True, padding="max_length")

# Tokenize the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Load model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

In [ ]:
from transformers import Trainer, TrainingArguments

from dreadnode.integrations.transformers import DreadnodeCallback
import dreadnode as dn

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=5,
    load_best_model_at_end=False,
    push_to_hub=False,
    run_name="distilbert-sst2-demo",
)

# Initialize Trainer with RiggingCallback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    callbacks=[DreadnodeCallback(project="training")],
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()